In [1]:
import pandas as pd
import numpy as np
import requests
import json
from collections import Counter

In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### CountList é uma função que retorna um dicionário contendo uma chave, que é o id de campeão e seu valor é a quantidade de vezes que esse campeão apareceu na lista.

In [3]:
def CountList(e):
    return Counter(e)
def Uniquefy(e):
    return list(set(e))

# Leitura dos dados

### Foi utilizado somente o dataset dos desafiantes, pois são os jogadores com mais habilidade e possivelmente com mais número de campeões utilizados.

In [4]:
champ = pd.read_csv('participants.csv')
champ = champ.loc[champ['gameDuration'] > 1500]
champ['played'] = 1

/home/neto/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Groupby do dataset

### O dataset é agrupado por Id do participante e contém uma lista de todos os campeões já jogador por aquele jogador.

In [5]:
gpby = champ.groupby(['participantId'])['championId'].apply(list).reset_index(name='championsPlayed')
gpby['count'] = gpby['championsPlayed'].apply(len)
gpby = gpby.loc[gpby['count'] > 5]

### Aplicamos o explode nessa lista de campeões para ter uma linha para cada campeão.

In [6]:
df_usable = gpby.explode('championsPlayed')
df_usable['played'] = 1
df_usable.drop('count',axis='columns', inplace=True)
df_usable

,participantId,championsPlayed,played
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,14,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,84,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,62,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,164,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,106,1
...,...,...,...
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,3,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,147,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1


### É aplicado outro groupby para somar quantas vezes cada campeão foi utiliado por aquele jogador.

In [7]:
df_grouped = df_usable.groupby(['participantId', 'championsPlayed'])['played'].sum()
cesta = df_grouped.unstack(fill_value=0)
cesta.reset_index().set_index('participantId')
df_grouped

participantId                                             championsPlayed
--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqCsNQ3IshLwd  6                  4
                                                          14                 2
                                                          24                 2
                                                          31                 2
                                                          39                 3
                                                                            ..
zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA           117                1
                                                          147                5
                                                          201                1
                                                          235                2
                                                          526                3
Name: played, Length: 14118, dtype: int64

# Apriori

### Esse groupby então é transformado em um dataset onde os campeões jogador por um dado participante são marcados com 1 e os não jogados marcados por 0.

In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

cesta = cesta.applymap(encode_units)
frequent_itemset = apriori(cesta, min_support=0.01, use_colnames=True)
cesta

championsPlayed,1,2,3,4,5,6,7,8,9,10,...,516,517,518,523,526,555,777,875,876,887
participantId,,,,,,,,,,,,,,,,,,,,,
--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqCsNQ3IshLwd,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
--bUxb3LAeED-fpHRWdWa-C1WRaCqP5MJV6wy5h65oSJ,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
-0Lkl8D8ElJldGfNYcUnVmkD0uGaFTfj9gfwJ5f50HV_guk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-0QvxtwHrJvG4nwAeZMO6yLPb-tUpSWVFrQ6OP-FyCcbs2E,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,1,0,0,0
-0q9CShTijqQFgPFXUabkwgN4SEYXlZhS_Zj8QTUxVQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zkoAqQNcV78IkbP6hOqQ07l9q1v-iJmz4a6FupZKz9UihOk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
zmk6Fd82FgJIZeWFdxIzbTw4nLrRcbIzrAYcaMoTQcVk8g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
znqUOlGVV0ouySxNrxJ6TGnTC-wGtx_ARiFcXsAQR_O2J1k,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modelo de recomendação
### o sistema receberá um vetor com os campeões já utilizados pelo usuário, para cada um deles será feita uma recomedação dos 5 melhores em support, após feito isso para todos os campeões enviados, as listas de recomedação são unificadas, retiradas as ocorrências de campeões já utilizados e então são ordenados por numero de ocorrências.

In [9]:
recommendList = []
testInput = [12, 111, 497]
for i in testInput:
    def compare (a):
        return a.__contains__(i)
    rules = association_rules(frequent_itemset,metric="confidence",min_threshold=0.01)
    rules_test = rules[rules['antecedents'].map(compare)]
    lists = rules_test.sort_values('support', ascending=False).head(5)['consequents'].tolist()
    mapItemset = map(list, lists)
    listItems = list(mapItemset)
    newList = []
    for l in listItems:
        newList.extend(l)
    recommendList.extend(newList)
recommendList = [s for s in recommendList if s not in testInput]
recommendCount = CountList(recommendList)

In [10]:
recommendCount

Counter({412: 3, 43: 3, 89: 3, 235: 2, 526: 1})

In [11]:
recommendCount = dict(sorted(recommendCount.items(), key=lambda item: item[1], reverse=True))

### É retornado para o usuário então uma lista com 5 personagens que mais tem a ver com o estilo de jogo do usuário.

In [12]:
list(recommendCount.keys())[0:5]

[412, 43, 89, 235, 526]